In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import numpy as np
import pandas as pd
from PIL import Image
import copy
import scipy.io

Function to load all images from the folder as a dataset

1) The len function in the dataset helps the torch dataloader to know how many items to iterate over for the whole dataset

2) The getitem function uses the value returned from the length and iterates over all the indexes from the value. *All this is used by the torch dataloader.*

In [0]:
class SunDataset(object):
    def __init__(self, image_list, labels):
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = image_list
        self.labels = labels

    def __getitem__(self, idx):
        # load images
        img = Image.open(self.imgs[idx])
        img = img.convert(mode='RGB')
        img = transforms.functional.resize(img, (400,400))
        img = transforms.functional.to_tensor(img)
        img = img.view(3,400,400)
        img = img/255
        label = self.labels[idx]

        return img, label

    def __len__(self):
        return len(self.imgs)

In [0]:
images = scipy.io.loadmat('gdrive/My Drive/Sun_Att/SUNAttributeDB/images.mat')
im_list = [list(images['images'][i][0])[0] for i in range(len(images['images']))]
im_path  = [os.path.join('gdrive/My Drive/Sun_Att/images', i) for i in im_list]

In [0]:
im_path[0:2]

['gdrive/My Drive/Sun_Att/images/a/abbey/sun_aakbdcgfpksytcwj.jpg',
 'gdrive/My Drive/Sun_Att/images/a/abbey/sun_aaoktempcmudsvna.jpg']

In [0]:
attributes = scipy.io.loadmat('gdrive/My Drive/Sun_Att/SUNAttributeDB/attributeLabels_continuous.mat')
labels = attributes['labels_cv']
labels = (labels>0).astype(int)
labels[0]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0])

In [0]:
#attributes = scipy.io.loadmat('gdrive/My Drive/Sun_Att/SUNAttributeDB/attributeLabels_continuous.mat')
#labels = attributes['labels_cv']
#labels = labels*100
#labels = labels.astype(int)
#labels[0]

In [0]:
#labels = np.where(labels==33, 1, labels)
#labels = np.where(labels==66, 2, labels)
#labels = np.where(labels==100, 3, labels)

In [0]:
labels[0]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0])

In [0]:
dataset = SunDataset(im_path, labels)

In [0]:
test_im, test_lb =dataset.__getitem__(0)

In [0]:
test_im.shape, test_lb.shape

(torch.Size([3, 400, 400]), (102,))

In [0]:
indices = torch.randperm(len(dataset)).tolist()
train_size = int(len(indices)*0.9)
test_size = int(len(indices)*0.1)
train_size, test_size

(12906, 1434)

In [0]:
train_dataset = torch.utils.data.Subset(dataset, indices[:train_size])
test_dataset = torch.utils.data.Subset(dataset, indices[-test_size:])

In [0]:
test_dataset.dataset.transform = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset.dataset.transform = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
#    transforms.RandomHorizontalFlip()
])

In [0]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=32, shuffle=True, num_workers=4)

test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [0]:
tmp_img, tmp_lbl = next(iter(train_loader))

In [0]:
tmp_lbl.float()

tensor([[0., 0., 0.,  ..., 1., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [0]:
tmp_lbl.shape
sum_lbl = torch.sum(tmp_lbl.data)
sum_lbl

tensor(471)

In [0]:
#model_ft = models.mobilenet_v2(pretrained=True)
model_ft = models.resnet50(pretrained=True)

In [0]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
#p_count = 0
for param in model_ft.parameters():
#    p_count += 1
#    if p_count > 141: #or p_count == 62:
        param.requires_grad = False
#    else:
#        param.requires_grad = False
#print(p_count)

In [0]:
num_ftrs = model_ft.fc.in_features
#model_ft.classifier[1] = nn.Linear(num_ftrs, len(labels[0]))

model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, len(labels[0])), nn.Sigmoid())

In [0]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
len(train_dataset)

12906

In [0]:
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        model.train()  # Set model to training mode

        running_loss = 0.0
        running_corrects = 0
        running_total = 0
        batch_loss = 0
        #batch_acc = 0
        batch = 0
        #batch_no = 1

            # Iterate over data.
        for inputs, labels in train_loader:
            batch = batch + 1
            inputs = inputs.to(device)
            labels = labels.float()
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                #print(outputs)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                loss.backward()
                optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                batch_loss += loss.item() * inputs.size(0)
                final_output = torch.where(outputs > 0.5, outputs, torch.zeros(1).to(device))
                final_output = torch.where(final_output > 0.5, torch.ones(1).to(device), torch.zeros(1).to(device))
                
                
                #running_corrects += torch.sum(final_output.data * labels.data)
                running_corrects += torch.sum(final_output.data == labels.data)
                running_total += 32 * 102 #torch.sum(labels.data)
                #batch_acc += running_corrects.double()/running_total.double()

            
            if batch % 50 == 0:
                #batch_loss = (running_loss/(32*50) - batch_loss)
                batch_loss = batch_loss/(32*50)
                print('Batch {}, Batch Loss: {:.4f}, Acc: {:.4f}'.format(batch, batch_loss, running_corrects.float() / torch.tensor(running_total).float()))
                #batch_acc = 0
                batch_loss = 0

                
        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.float() / torch.tensor(running_total).float()
        
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
            
        
        model.eval()
        vrunning_loss = 0
        vrunning_corrects = 0
        vrunning_total = 0
        for vinputs, vlabels in test_loader:
            vinputs = vinputs.to(device)
            vlabels = vlabels.float().to(device)
            with torch.no_grad():
                voutputs = model(vinputs)
                vloss = criterion(voutputs, vlabels)
                vrunning_loss += vloss.item() * vinputs.size(0)
            vfinal_output = torch.where(voutputs > 0.5, voutputs, torch.zeros(1).to(device))
            vfinal_output = torch.where(vfinal_output > 0.5, torch.ones(1).to(device), torch.zeros(1).to(device))
            
            
            vrunning_corrects += torch.sum(vfinal_output.data == vlabels.data)
            vrunning_total += 32 * 102 #torch.sum(vlabels.data)
            
        vepoch_loss = vrunning_loss / len(test_dataset)
        vepoch_acc = vrunning_corrects.float() / torch.tensor(vrunning_total).float()
            
        print('{} Epoch Loss: {:.4f} Acc: {:.4f}'.format(epoch, epoch_loss, epoch_acc))
        print('Validation: {} Epoch Val Loss: {:.4f} Val Acc: {:.4}'.format(epoch, vepoch_loss, vepoch_acc))

        print()
        scheduler.step()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

criterion = nn.BCELoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.1, momentum=0.9) #Best 015, 091

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=2, gamma=0.01)

In [0]:
model_res = train_model(model_ft, train_loader, test_loader, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)

Epoch 0/4
----------
Batch 50, Batch Loss: 0.3760, Acc: 0.8590
Batch 100, Batch Loss: 0.3035, Acc: 0.8704
Batch 150, Batch Loss: 0.2844, Acc: 0.8757
Batch 200, Batch Loss: 0.2763, Acc: 0.8788
Batch 250, Batch Loss: 0.2716, Acc: 0.8810
Batch 300, Batch Loss: 0.2665, Acc: 0.8828
Batch 350, Batch Loss: 0.2671, Acc: 0.8840
Batch 400, Batch Loss: 0.2604, Acc: 0.8852
0 Epoch Loss: 0.2880 Acc: 0.8838
Validation: 0 Epoch Val Loss: 0.2547 Val Acc: 0.8921

Epoch 1/4
----------
Batch 50, Batch Loss: 0.2556, Acc: 0.8957
Batch 100, Batch Loss: 0.2544, Acc: 0.8959
Batch 150, Batch Loss: 0.2514, Acc: 0.8963
Batch 200, Batch Loss: 0.2542, Acc: 0.8963
Batch 250, Batch Loss: 0.2510, Acc: 0.8966
Batch 300, Batch Loss: 0.2482, Acc: 0.8968
Batch 350, Batch Loss: 0.2519, Acc: 0.8968
Batch 400, Batch Loss: 0.2468, Acc: 0.8971
1 Epoch Loss: 0.2517 Acc: 0.8956
Validation: 1 Epoch Val Loss: 0.2447 Val Acc: 0.8968

Epoch 2/4
----------
Batch 50, Batch Loss: 0.2442, Acc: 0.9002
Batch 100, Batch Loss: 0.2473, Acc:

In [0]:
test_im_batch, test_lbl_batch = next(iter(test_loader))

In [0]:
test_im = test_im_batch[0].to(device)
test_im = test_im.unsqueeze(0)
test_lbl = test_lbl_batch[0].to(device).float()

In [0]:
test_output = model_ft(test_im)

In [0]:
test_output

tensor([[0.8900, 0.1021, 0.0888, 0.3151, 0.1428, 0.3892, 0.0370, 0.2458, 0.0131,
         0.0676, 0.0118, 0.0441, 0.0237, 0.8137, 0.8371, 0.4972, 0.0182, 0.0424,
         0.0324, 0.0320, 0.0232, 0.0629, 0.0617, 0.0682, 0.0442, 0.0160, 0.0196,
         0.0065, 0.0195, 0.0075, 0.0287, 0.0776, 0.0353, 0.0371, 0.0118, 0.0055,
         0.0395, 0.0526, 0.0370, 0.0116, 0.0271, 0.0081, 0.0131, 0.0157, 0.0135,
         0.0191, 0.0089, 0.1050, 0.1053, 0.0101, 0.0254, 0.0124, 0.0321, 0.1495,
         0.0812, 0.0253, 0.0391, 0.0553, 0.0823, 0.1168, 0.1132, 0.0830, 0.0461,
         0.0318, 0.0510, 0.6719, 0.8917, 0.4672, 0.7411, 0.5736, 0.5257, 0.5611,
         0.0383, 0.0216, 0.8942, 0.3804, 0.0380, 0.0553, 0.1509, 0.1262, 0.0675,
         0.7459, 0.0865, 0.0384, 0.0505, 0.2587, 0.7596, 0.4733, 0.4082, 0.9013,
         0.0435, 0.0506, 0.6528, 0.3698, 0.3331, 0.0816, 0.1258, 0.0715, 0.0247,
         0.1296, 0.4192, 0.0767]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [0]:
test_lbl

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')

In [0]:
m_o = torch.where(test_output > 0.5, test_output, torch.zeros(1).to(device))
m_o = torch.where(m_o > 0.5, torch.ones(1).to(device), torch.zeros(1).to(device))

In [0]:
m_o

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0')

In [0]:
correct = (m_o.data == test_lbl.data)

In [0]:
correct

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True, False, False,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True, False,  True,  True,  True,  True,
          True, False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True, False,  True,  True,
          True,  True, False, False,  True,  True,  True,  True,  True,  True,
          True,  True]], device='cuda:0')

In [0]:
torch.sum(correct.data)

tensor(95, device='cuda:0')

In [0]:
del_var = torch.sum(correct.data)
del_var/torch.tensor(102).float()

tensor(0.9314, device='cuda:0')